## Linear regresion on data for device id 249
Here I will try to find the function to fit the data and detect anomalies

In [1]:
import math
import pandas as pandas
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from statistics import mean
from numpy.random import shuffle

import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt

data_248 = pandas.read_json("../data/device_id_248.json")

In [2]:
# Already prepared data from Intial data analysis - removed empty rows and wrong dates
data_248

,tot1,tot2,analog2
2018-12-11 12:10:00,116.15,0.00,0.82
2018-12-11 12:11:00,118.02,0.00,0.82
2018-12-11 12:12:00,119.90,0.00,0.82
2018-12-11 12:13:00,121.77,0.00,0.81
2018-12-11 12:14:00,123.64,0.00,0.82
...,...,...,...
2020-04-01 14:05:00,977428.25,-4713.21,1.54
2020-04-01 14:06:00,977428.88,-4713.21,1.54
2020-04-01 14:07:00,977429.50,-4713.21,1.54
2020-04-01 14:08:00,977430.12,-4713.21,1.54


Function generate data for a given timeframe

In [28]:
def generate_day_view(start_date, end_date, data):
    if len(data.index) < 1:
        raise Exception("No data for this date")
    
    start_day = start_date.strip() + " 00:00"
    end_day = end_date.strip() + " 23:59"
    data = data.loc[(data.index >= start_day) & (data.index <= end_day)]
    
    timestamp = data.index.values
    # y_axis = data[["analog2"]].to_numpy().ravel()
    
    # all values without pressure -> [:,:3], just pressure -> [:,3:]
    data.index = pandas.to_datetime(data.index)
    matrix_X = data.reset_index().values

    # matrix_X["index"] = pandas.to_datetime(matrix_X['index'])
    # matrix_X = data.reset_index().values
    
    # learn_Y = data.reset_index().values[:,3:]
    size_of_learn_set = round(len(matrix_X) * 0.70)
    
    learn_set = matrix_X[:size_of_learn_set]
    test_set  = matrix_X[size_of_learn_set:]
    
    model = LinearRegression()
    # model.fit(matrix_X[:,:3], matrix_X[:,3:])
    # predicted_values = model.predict(test_set)
    
    return timestamp


    """timestamp = timestamp.tolist()
    common = []
    for i in range(0, len(y_axis)):
        common.append([timestamp[i], y_axis[i]])

    combined = np.vstack((timestamp.tolist(), y_axis))
    return common"""

In [29]:
generate_day_view("2020-01-01", "2020-04-01", data_248)

array(['2020-01-01T00:00:00.000000000', '2020-01-01T00:01:00.000000000',
       '2020-01-01T00:02:00.000000000', ...,
       '2020-04-01T14:07:00.000000000', '2020-04-01T14:08:00.000000000',
       '2020-04-01T14:09:00.000000000'], dtype='datetime64[ns]')

In [5]:
# Napovedana vrednost - aka tuki smo iskali oceno, na zacetku smo dali ocene vseh filmov in uporabnika, 
# pol smo pa dali uporabnika in ocene in je probalo napovedat

"""
Napovedat hocmo pritisk torej je pritisk to, napovedat ga hocmo na podlagi parametru [time, tot1, tot2]
- torej rabim za vsak pritisk matriko X [[time, tot1, tot2], [time, tot1, tot2], [time, tot1, tot2], ......]
- Y bodo pa pritiski [1.5, 1.5, 1.6, ....]
- za predictat pa vzamem spet matriko X -  [[time, tot1, tot2],... ] za en dan... -> vrne mi vrednosti
- to pol comparam z dejanskim pritiskom od takrat in zracunam razliko med napovedanim
"""

rest_matrix =  rest_matrix[user_vector > 0]
user_vector = user_vector[user_vector > 0]   # vrzem vn tiste ki si jih ni ogledu

learn_len = round(len(user_vector) * 0.75)
user_vector_learn = user_vector[:learn_len]
user_vector_test  = user_vector[learn_len:]

learn_set = rest_matrix[:learn_len]
test_set  = rest_matrix[learn_len:]

model = LinearRegression()
model.fit(learn_set, user_vector_learn)
predicted_values = model.predict(test_set)

accuracy = mean_squared_error(predicted_values, user_vector_test)
accuracy_arr.append(accuracy)



NameError: name 'rest_matrix' is not defined

In [ ]:
fig = plt.figure(figsize=(8, 6), dpi= 100, facecolor='w')
plt.plot(timestamp, y_axis, color="blue")
plt.margins(0, 0.1)

plt.ylabel('Pressure in bar ?')
plt.xlabel('Date')
plt.title('Time')
# plt.savefig('blue_id.png')
plt.show()

In [ ]:

"""learn_set_num = round(len(data_set_1) * 0.70)

learn_set = data_set_1[:learn_set_num]
test_set  = data_set_1[learn_set_num:]
model = LinearRegression()

model.fit([[i[0]] for i in learn_set], [i[1] for i in learn_set])
predicted_values = model.predict([[i[1]] for i in test_set])""";